In [10]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from transformers import AutoConfig, AutoTokenizer, AutoModelWithLMHead
import torch

In [8]:
model_name_or_path='./debiased_models/bert/original-checkpoint-best'
cache_dir=''

config=AutoConfig.from_pretrained(model_name_or_path, cache_dir=cache_dir)
tokenizer=AutoTokenizer.from_pretrained(model_name_or_path, cache_dir=cache_dir)

block_size=tokenizer.model_max_length

model=AutoModelWithLMHead.from_pretrained(model_name_or_path, from_tf=bool(".ckpt" in model_name_or_path), config=config, cache_dir=cache_dir)

if tokenizer._pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<pad>'})
    model.resize_token_embeddings(len(tokenizer))


In [9]:
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [18]:
with open('./attribute_target_words/targets/gender_occupation.txt') as f:
    text=f.read().splitlines()

In [ ]:
def tsne(vecs, labels, title="", ind2label=None, words=None, metric="l2"):
    tsne=TSNE(n_components=2)
    vecs_2d=tsne.fit_transform(vecs)
    label_names=sorted(list(set(labels.tolist())))
    num_labels=len(label_names)

    names=sorted(set(labels.tolist()))

    plt.figure(figsize=(6,5))
    colors="red", "blue"
    for i, c, label in zip(sorted(set(labels.tolist())), colors, names):
        plt.scatter(vecs_2d[labels==i, 0], vecs_2d[labels==i,], c=c, label=label if ind2label is None else ind2label[label], alpha = 0.3, marker = "s" if i==0 else "o")
        plt.legend(loc="upper right")

    plt.title(title)
    #plt.savefig("embeddings.{}.png".format(title), dpi=600)
    plt.show()
    return vecs_2d